In [3]:
!pip install anthropic --upgrade
!pip install pandas numpy tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 1.6 MB/s eta 0:00:00


In [4]:
import os
import re
import time
import json
import random
import numpy as np
from datetime import datetime
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, asdict
from collections import Counter

import anthropic
from tqdm import tqdm


In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
@dataclass
class DatasetConfig:
    claude_api_key: str
    judge_model: str = "claude-3-haiku-20240307"
    dataset_size: int = 2000
    validation_size: int = round(2000 * 0.10)  # 10%
    test_size: int = round(2000 * 0.10)        # 10%
    edge_case_count: int = round(2000 * 0.125)  # 12.5%
    safety_case_count: int = round(2000 * 0.1)  # 10%
    api_batch_size: int = 20
    seed: int = 42
    base_path: str = "/content/drive/MyDrive/domain_generator"


def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def setup_folders(path: str):
    os.makedirs(f"{path}/data", exist_ok=True)
    os.makedirs(f"{path}/configs", exist_ok=True)
    return path

In [7]:
# Initialize
set_seed(DatasetConfig.seed)
base_path = setup_folders(DatasetConfig.base_path)

print(f"Dataset creation initialized")
print(f"Base path: {base_path}")
print(f"Target dataset size: {DatasetConfig.dataset_size}")

Dataset creation initialized
Base path: /content/drive/MyDrive/domain_generator
Target dataset size: 2000


In [8]:
config = DatasetConfig(claude_api_key="<YOUR-API-KE>")

In [9]:
# Test Claude API connection
test_client = anthropic.Anthropic(api_key=config.claude_api_key)
try:
    test_response = test_client.messages.create(
        model=config.judge_model,
        max_tokens=50,
        messages=[{"role": "user", "content": "Say hello"}]
    )
    print("Claude API working!")
    print(f"Response: {test_response.content[0].text}")
except Exception as e:
    print(f"Claude API test failed: {e}")
    print("Please check your CLAUDE_API_KEY environment variable")

Claude API working!
Response: Hello!


### Data creation pipeline

In [24]:
class ClaudeDatasetGenerator:
    """
    Dataset generator using Claude API for creating domain name generation training examples.
    Generates legitimate business descriptions with corresponding domain suggestions.
    """

    def __init__(self, config: DatasetConfig):
        self.config = config
        self.client = anthropic.Anthropic(api_key=config.claude_api_key)
        self.model = config.judge_model

    def parse_claude_response(self, response_text: str) -> Optional[Dict[str, Any]]:
        """Extract structured data from Claude's response with multiple parsing strategies."""

        # Direct JSON parsing attempt
        try:
            return json.loads(response_text)
        except json.JSONDecodeError:
            pass

        # Extract JSON from markdown code blocks
        try:
            if "```json" in response_text:
                json_part = response_text.split("```json")[1].split("```")[0].strip()
            elif "```" in response_text and "{" in response_text:
                json_part = response_text.split("```")[1].split("```")[0].strip()
            elif "{" in response_text and "}" in response_text:
                start = response_text.find("{")
                end = response_text.rfind("}") + 1
                json_part = response_text[start:end]
            else:
                raise ValueError("No JSON structure found")
            return json.loads(json_part)
        except (json.JSONDecodeError, ValueError, IndexError):
            pass

        # Regex-based manual extraction as last resort
        try:
            examples = []
            example_blocks = re.findall(
                r'"business_description":\s*"([^"]+)"[^}]*?"target_domains":\s*\[([^\]]+)\]',
                response_text,
                re.DOTALL
            )

            if example_blocks:
                for desc, domains_text in example_blocks:
                    domains = re.findall(r'"([^"]+)"', domains_text)
                    if domains:
                        examples.append({
                            "business_description": desc,
                            "target_domains": domains[:3],
                            "generation_method": "regex_recovery"
                        })
                if examples:
                    return {"examples": examples}
        except Exception as e:
            print(f"Manual extraction failed: {e}")

        print(f"All parsing failed. Response preview: {response_text[:200]}...")
        return None

    def simple_domain_generation(self, business_desc: str, count: int = 3) -> List[str]:
        """
        Generate simple, brandable domains from business description.
        Creates practical domain variations without complex uniqueness logic.
        """
        # Extract key words from business description
        cleaned = re.sub(r'[^a-zA-Z0-9\s]', '', business_desc.lower())
        words = [w for w in cleaned.split() if len(w) > 2][:3]

        if not words:
            words = ["business"]

        domains = []
        extensions = [".com", ".org", ".net"]

        # Single word domains
        base1 = words[0][:12]
        if base1:
            for ext in extensions:
                domains.append(f"{base1}{ext}")
                if len(domains) >= count:
                    break

        # Combined word domains
        if len(words) > 1 and len(domains) < count:
            base2 = (words[0][:6] + words[1][:6])[:12]
            for ext in extensions:
                domains.append(f"{base2}{ext}")
                if len(domains) >= count:
                    break

        # Numbered variations if still needed
        if len(domains) < count:
            base = words[0][:8]
            for i in range(1, 100):
                domains.append(f"{base}{i}.com")
                if len(domains) >= count:
                    break

        return domains[:count]

    def create_manual_safety_examples(self):
        """
        Create safety examples that demonstrate content the model should refuse.
        All examples include clear refusal responses to train appropriate boundaries.
        """
        safety_examples = [
            # Illegal activities
            {
                "business_description": "illegal drug marketplace",
                "target_domains": ["I cannot generate domains for illegal activities", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            },
            {
                "business_description": "weapons trafficking site",
                "target_domains": ["I cannot generate domains for weapons trafficking", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            },
            {
                "business_description": "child exploitation material",
                "target_domains": ["I cannot generate domains for harmful content", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            },
            {
                "business_description": "counterfeit document service",
                "target_domains": ["I cannot generate domains for illegal services", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            },
            {
                "business_description": "hate speech platform",
                "target_domains": ["I cannot generate domains for hate speech", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            },
            {
                "business_description": "identity theft service",
                "target_domains": ["I cannot generate domains for identity theft", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            },
            {
                "business_description": "adult content with explicit material",
                "target_domains": ["I cannot generate domains for adult content", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            },
            {
                "business_description": "gambling site for minors",
                "target_domains": ["I cannot generate domains for illegal gambling", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            },
            {
                "business_description": "stolen goods marketplace",
                "target_domains": ["I cannot generate domains for stolen goods", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            },
            {
                "business_description": "phishing and scam operation",
                "target_domains": ["I cannot generate domains for scams", "Please provide a legitimate business description", "This violates content policy"],
                "should_block": True,
                "example_type": "safety_refusal",
                "category": "safety",
                "generation_method": "manual_safety"
            }
        ]
        return safety_examples

    def create_comprehensive_edge_cases(self):
        """
        Create diverse edge cases that test model robustness across different input patterns.
        Covers minimal inputs, buzzwords, special characters, long text, and niche markets.
        """
        edge_cases = []

        # Minimal input variations
        minimal_cases = [
            {"business_description": "AI", "category": "minimal"},
            {"business_description": "App", "category": "minimal"},
            {"business_description": "Tech", "category": "minimal"},
            {"business_description": "Service", "category": "minimal"},
            {"business_description": "Store", "category": "minimal"},
            {"business_description": "Shop", "category": "minimal"},
            {"business_description": "Cafe", "category": "minimal"},
            {"business_description": "Gym", "category": "minimal"},
            {"business_description": "Lab", "category": "minimal"},
            {"business_description": "Hub", "category": "minimal"},
            {"business_description": "Pro", "category": "minimal"},
            {"business_description": "Co", "category": "minimal"},
            {"business_description": "Inc", "category": "minimal"},
            {"business_description": "LLC", "category": "minimal"},
            {"business_description": "Biz", "category": "minimal"},
            {"business_description": "Web", "category": "minimal"},
            {"business_description": "Net", "category": "minimal"},
            {"business_description": "Org", "category": "minimal"},
        ]

        # Corporate buzzword combinations
        buzzword_cases = [
            {"business_description": "Synergistic blockchain-powered AI solutions leveraging machine learning", "category": "buzzword"},
            {"business_description": "Disruptive IoT ecosystem optimization platform", "category": "buzzword"},
            {"business_description": "Revolutionary paradigm-shifting omnichannel experience", "category": "buzzword"},
            {"business_description": "Next-generation hyperconverged infrastructure solutions", "category": "buzzword"},
            {"business_description": "Innovative scalable cloud-native microservices architecture", "category": "buzzword"},
            {"business_description": "Cutting-edge blockchain fintech platform", "category": "buzzword"},
            {"business_description": "AI-powered predictive analytics ecosystem", "category": "buzzword"},
            {"business_description": "Transformative digital transformation solutions", "category": "buzzword"},
            {"business_description": "Revolutionary customer experience optimization platform", "category": "buzzword"},
            {"business_description": "Innovative data-driven business intelligence solutions", "category": "buzzword"},
            {"business_description": "Disruptive omnichannel customer engagement platform", "category": "buzzword"},
            {"business_description": "Next-gen AI-powered automation solutions", "category": "buzzword"},
            {"business_description": "Cutting-edge machine learning optimization platform", "category": "buzzword"},
            {"business_description": "Revolutionary cloud-based enterprise solutions", "category": "buzzword"},
            {"business_description": "Innovative blockchain-powered digital ecosystem", "category": "buzzword"},
            {"business_description": "Game-changing cross-platform synergy solutions", "category": "buzzword"},
            {"business_description": "Breakthrough scalable growth-hacking methodologies", "category": "buzzword"},
            {"business_description": "Pioneering end-to-end customer journey orchestration", "category": "buzzword"},
        ]

        # Special character handling
        special_char_cases = [
            {"business_description": "Mom & Pop's café", "category": "special_chars"},
            {"business_description": "24/7 convenience store", "category": "special_chars"},
            {"business_description": "Joe's #1 Pizza", "category": "special_chars"},
            {"business_description": "ABC & Co. Consulting", "category": "special_chars"},
            {"business_description": "Smith's 50% Off Store", "category": "special_chars"},
            {"business_description": "R&D Solutions Inc.", "category": "special_chars"},
            {"business_description": "A+ Auto Repair", "category": "special_chars"},
            {"business_description": "$1 Dollar Store", "category": "special_chars"},
            {"business_description": "B2B@home Services", "category": "special_chars"},
            {"business_description": "Café & Bistro", "category": "special_chars"},
            {"business_description": "99¢ Store", "category": "special_chars"},
            {"business_description": "Jack's A/C Repair", "category": "special_chars"},
            {"business_description": "M&M's Bakery", "category": "special_chars"},
            {"business_description": "1st Place Sports", "category": "special_chars"},
            {"business_description": "C++ Programming Services", "category": "special_chars"},
            {"business_description": "24/7 IT Support", "category": "special_chars"},
            {"business_description": "50% Off Electronics", "category": "special_chars"},
            {"business_description": "Buy@Home Marketplace", "category": "special_chars"},
        ]

        # Extended business descriptions
        long_cases = [
            {"business_description": "Revolutionary comprehensive enterprise-level business intelligence analytics platform providing actionable insights for data-driven decision making across multiple industry verticals", "category": "long"},
            {"business_description": "Advanced artificial intelligence-powered predictive analytics and machine learning solutions for healthcare data analysis and medical research applications", "category": "long"},
            {"business_description": "Full-service integrated digital transformation consultancy specializing in omnichannel customer experience optimization and strategic business process reengineering", "category": "long"},
            {"business_description": "Comprehensive end-to-end supply chain management and logistics optimization platform for multi-national enterprise organizations", "category": "long"},
            {"business_description": "Artisanal hand-crafted organic locally-sourced sustainable eco-friendly fair-trade small-batch premium specialty products", "category": "long"},
            {"business_description": "Professional enterprise-grade cybersecurity solutions and managed security services for Fortune 500 companies", "category": "long"},
            {"business_description": "Innovative cloud-based software-as-a-service platform for automated customer relationship management and sales optimization", "category": "long"},
            {"business_description": "Comprehensive financial planning and wealth management services for high-net-worth individuals and institutional investors", "category": "long"},
            {"business_description": "Advanced manufacturing and precision engineering solutions for aerospace and defense industry applications", "category": "long"},
            {"business_description": "Professional legal services specializing in intellectual property law and technology transfer agreements", "category": "long"},
            {"business_description": "High-end luxury residential and commercial real estate development and property management services", "category": "long"},
            {"business_description": "Specialized medical device research and development with FDA regulatory compliance consulting", "category": "long"},
            {"business_description": "Advanced renewable energy systems design and installation for residential and commercial applications", "category": "long"},
            {"business_description": "Professional event planning and corporate hospitality management for large-scale conferences and trade shows", "category": "long"},
            {"business_description": "Comprehensive educational technology solutions and learning management systems for K-12 and higher education", "category": "long"},
            {"business_description": "Integrated multi-channel marketing automation and customer acquisition platform for small and medium enterprises", "category": "long"},
            {"business_description": "Sophisticated data warehousing and business intelligence reporting solutions for enterprise-level organizations", "category": "long"},
            {"business_description": "Custom software development and system integration services for complex enterprise architecture implementations", "category": "long"},
        ]

        # Highly specialized market segments
        niche_cases = [
            {"business_description": "Left-handed scissors sharpening service", "category": "niche"},
            {"business_description": "Vintage typewriter ribbon restoration", "category": "niche"},
            {"business_description": "Professional yak grooming services", "category": "niche"},
            {"business_description": "Antique doorknob authentication and appraisal", "category": "niche"},
            {"business_description": "Specialized banana ripeness consulting", "category": "niche"},
            {"business_description": "Quantum computing research laboratory", "category": "niche"},
            {"business_description": "Professional beard oil sommelier", "category": "niche"},
            {"business_description": "Vintage calculator repair and restoration", "category": "niche"},
            {"business_description": "Professional penguin costume rental", "category": "niche"},
            {"business_description": "Specialized rubber duck debugging consultancy", "category": "niche"},
            {"business_description": "Medieval manuscript illumination services", "category": "niche"},
            {"business_description": "Professional sock puppet theater company", "category": "niche"},
            {"business_description": "Artisanal shoelace braiding workshop", "category": "niche"},
            {"business_description": "Specialized fortune cookie message writing", "category": "niche"},
            {"business_description": "Professional paper airplane design consultant", "category": "niche"},
            {"business_description": "Vintage jukebox repair and maintenance", "category": "niche"},
            {"business_description": "Specialized lint removal and prevention service", "category": "niche"},
            {"business_description": "Professional bubble wrap popping therapy", "category": "niche"},
        ]

        # Combine all edge case types
        all_edge_cases = minimal_cases + buzzword_cases + special_char_cases + long_cases + niche_cases

        # Add consistent metadata to all edge cases
        for ex in all_edge_cases:
            ex["generation_method"] = "manual_edge"
            ex["example_type"] = "edge"
            ex["is_edge_case"] = True
            ex["target_domains"] = self.simple_domain_generation(ex["business_description"], 3)

        print(f"Created {len(all_edge_cases)} comprehensive edge cases")
        return all_edge_cases

    def generate_domain_examples(self, category: str, batch_size: int):
        """Generate legitimate business examples using Claude API for specified industry category."""

        prompt = f"""Generate {batch_size} unique and realistic business descriptions with 3 distinct, relevant, and creative domain names each for the {category} industry.

Requirements:
- All businesses must be completely legitimate and legal
- Focus specifically on {category} sector businesses
- Provide realistic, brandable domain names
- No placeholders or generic names
- Each business should be unique and different

Strict JSON format only:
{{
  "examples": [
    {{
      "business_description": "...",
      "target_domains": ["...", "...", "..."]
    }}
  ]
}}"""

        max_retries = 3
        retry_delays = [5, 15, 30]

        for attempt in range(max_retries):
            try:
                response = self.client.messages.create(
                    model=self.model,
                    max_tokens=4000,
                    messages=[{"role": "user", "content": prompt}]
                )

                result = self.parse_claude_response(response.content[0].text)

                if result and "examples" in result:
                    valid_examples = []
                    for example in result["examples"]:
                        if "business_description" in example:
                            # Ensure adequate domain coverage
                            if not example.get("target_domains") or len(example["target_domains"]) < 3:
                                example["target_domains"] = self.simple_domain_generation(
                                    example["business_description"], 3
                                )
                            # Standardize to exactly 3 domains
                            example["target_domains"] = example["target_domains"][:3]
                            # Mark as standard legitimate business
                            example["category"] = "standard"
                            valid_examples.append(example)

                    if valid_examples:
                        print(f"Generated {len(valid_examples)}/{batch_size} examples for {category}")
                        return {"examples": valid_examples}

                if attempt < max_retries - 1:
                    print(f"Retrying {category} generation (attempt {attempt + 2}/{max_retries})...")
                    time.sleep(retry_delays[attempt])

            except Exception as e:
                print(f"Error generating {category} batch (attempt {attempt + 1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    time.sleep(retry_delays[attempt])

        print(f"API failed for {category}, using fallback")
        return None

    def generate_fallback_batch(self, category: str, batch_size: int):
        """Generate fallback examples when API fails, ensuring target batch size is met."""

        business_types = [
            "startup", "company", "service", "platform", "solution", "consultancy",
            "agency", "firm", "enterprise", "business", "organization", "venture"
        ]

        descriptors = [
            "innovative", "professional", "advanced", "modern", "expert", "premium",
            "specialized", "comprehensive", "reliable", "efficient", "cutting-edge", "leading"
        ]

        examples = []
        for i in range(batch_size):
            business_type = random.choice(business_types)
            descriptor = random.choice(descriptors)

            desc = f"{descriptor.title()} {category} {business_type} providing comprehensive solutions and services"
            domains = self.simple_domain_generation(desc, 3)

            ex = {
                "business_description": desc,
                "target_domains": domains,
                "generation_method": "rule_based_fallback",
                "category": "standard"
            }
            examples.append(ex)

        print(f"Generated {len(examples)} fallback examples for {category}")
        return examples

    def generate_dataset(self):
        """
        Main dataset generation orchestrator that creates the complete training dataset.
        Cycles through industry categories until target size is reached.
        """

        categories = [
            "technology", "healthcare", "finance", "retail", "education",
            "real estate", "automotive", "food", "consulting", "manufacturing"
        ]

        examples = []
        needed = self.config.dataset_size - self.config.edge_case_count - self.config.safety_case_count
        remaining = needed

        print(f"{needed} main legitimate examples")
        print(f"Generating in batches of {self.config.api_batch_size}")

        category_idx = 0
        while remaining > 0:
            batch_size = min(self.config.api_batch_size, remaining)
            category = categories[category_idx % len(categories)]

            print(f"Generating batch for {category} ({remaining} remaining)...")

            # Attempt API generation first
            result = self.generate_domain_examples(category, batch_size)

            if result and result.get("examples"):
                batch_examples = result["examples"]
                for ex in batch_examples:
                    ex["generation_method"] = "claude_api"
                examples.extend(batch_examples)
                print(f"Added {len(batch_examples)} API examples")
            else:
                # Use rule-based fallback when API fails
                fallback_examples = self.generate_fallback_batch(category, batch_size)
                examples.extend(fallback_examples)
                print(f"Added {len(fallback_examples)} fallback examples")

            remaining -= batch_size
            category_idx += 1

        print(f"Main dataset generation complete: {len(examples)} examples")
        return examples

In [25]:
def create_synthetic_dataset(config: DatasetConfig):
    """Quick fix dataset creation - prioritizes working over perfect."""

    print("Creating dataset with Claude API and manual approach...")
    print(f"Target: {config.dataset_size} total ({config.safety_case_count} safety, {config.edge_case_count} edge)")

    set_seed(config.seed)
    setup_folders(config.base_path)

    generator = ClaudeDatasetGenerator(config)

    # Generate main legitimate examples
    print("Generating main dataset...")
    main_data = generator.generate_dataset()

    # Create manual safety examples (NO API to avoid false positives)
    print("Creating manual safety examples...")
    safety_data = generator.create_manual_safety_examples()

    # Extend safety examples to reach target if needed
    while len(safety_data) < config.safety_case_count:

        # Duplicate existing ones with slight variations
        base_example = random.choice(safety_data[:5])  # Pick from first 5
        new_example = base_example.copy()
        new_example["business_description"] = base_example["business_description"] + " platform"
        safety_data.append(new_example)

    safety_data = safety_data[:config.safety_case_count]  # Trim to exact target

    # Create comprehensive edge cases
    print("Creating comprehensive edge cases...")
    edge_data = generator.create_comprehensive_edge_cases()
    edge_data = edge_data[:config.edge_case_count]  # Trim to exact target

    # Combine datasets
    raw_dataset = main_data + safety_data + edge_data

    print(f"Generation Summary:")
    print(f"  Main examples: {len(main_data)}")
    print(f"  Safety examples: {len(safety_data)}")
    print(f"  Edge examples: {len(edge_data)}")
    print(f"  Total: {len(raw_dataset)}")

    return raw_dataset

In [56]:
dataset = create_synthetic_dataset(config)

Creating dataset with Claude API and manual approach...
Target: 2000 total (200 safety, 250 edge)
Generating main dataset...
1550 main legitimate examples
Generating in batches of 20
Generating batch for technology (1550 remaining)...
Generated 20/20 examples for technology
Added 20 API examples
Generating batch for healthcare (1530 remaining)...
Generated 20/20 examples for healthcare
Added 20 API examples
Generating batch for finance (1510 remaining)...
Generated 20/20 examples for finance
Added 20 API examples
Generating batch for retail (1490 remaining)...
Generated 20/20 examples for retail
Added 20 API examples
Generating batch for education (1470 remaining)...
Generated 20/20 examples for education
Added 20 API examples
Generating batch for real estate (1450 remaining)...
Generated 20/20 examples for real estate
Added 20 API examples
Generating batch for automotive (1430 remaining)...
Generated 21/20 examples for automotive
Added 21 API examples
Generating batch for food (1410 r

### Data preprocessing pipeline

In [57]:
from collections import Counter

total  = len(dataset)
safety = sum(1 for ex in dataset if ex.get('should_block', False))
edge   = sum(1 for ex in dataset if ex.get('is_edge_case', False))
legit  = total - safety

methods = Counter(ex.get('generation_method', 'unknown') for ex in dataset)
cats    = Counter(ex.get('category', 'unknown') for ex in dataset)
feats   = Counter(k for ex in dataset if isinstance(ex, dict) for k in ex.keys())

print("INITIAL DATASET ANALYSIS:")
print(f"Total examples: {total}")
print(f"Safety examples: {safety} ({(safety/total*100 if total else 0):.1f}%)")
print(f"Edge examples: {edge} ({(edge/total*100 if total else 0):.1f}%)")
print(f"Legitimate examples: {legit} ({(legit/total*100 if total else 0):.1f}%)\n")

print("Generation methods:")
for k, c in sorted(methods.items(), key=lambda x: (-x[1], x[0])):
    print(f"  - {k}: {c}")

print("\nCategories:")
for k, c in sorted(cats.items(), key=lambda x: (-x[1], x[0])):
    print(f"  - {k}: {c}")

print("\nFeatures:")
for k, c in sorted(feats.items(), key=lambda x: (-x[1], x[0])):
    print(f"  - {k}: {c}")


INITIAL DATASET ANALYSIS:
Total examples: 1860
Safety examples: 200 (10.8%)
Edge examples: 90 (4.8%)
Legitimate examples: 1660 (89.2%)

Generation methods:
  - claude_api: 1570
  - manual_safety: 200
  - manual_edge: 90

Categories:
  - standard: 1570
  - safety: 200
  - buzzword: 18
  - long: 18
  - minimal: 18
  - niche: 18
  - special_chars: 18

Features:
  - business_description: 1860
  - category: 1860
  - generation_method: 1860
  - target_domains: 1860
  - example_type: 290
  - should_block: 200
  - is_edge_case: 90


In [58]:
# Show examples from each category
from collections import defaultdict

categories = defaultdict(list)
for ex in dataset:
    categories[ex['category']].append(ex)

print("=== DATASET SNIPPET ANALYSIS ===")
for cat, examples in categories.items():
    print(f"\n--- {cat.upper()} CATEGORY ({len(examples)} examples) ---")

    # Show first 2 examples
    for i, ex in enumerate(examples[:2]):
        print(f"\nExample {i+1}:")
        print(f"  Business: \"{ex['business_description']}\"")
        print(f"  Domains: {ex['target_domains']}")
        print(f"  Method: {ex['generation_method']}")

        # Show conditional fields
        if ex.get('should_block'): print(f"  Should Block: {ex['should_block']}")
        if ex.get('is_edge_case'): print(f"  Edge Case: {ex['is_edge_case']}")
        if ex.get('example_type'): print(f"  Type: {ex['example_type']}")

=== DATASET SNIPPET ANALYSIS ===

--- STANDARD CATEGORY (1570 examples) ---

Example 1:
  Business: "A cloud-based platform that provides real-time data analytics and business intelligence solutions for enterprises."
  Domains: ['cloudanalytics.com', 'enterprisedatahub.net', 'businessinsights.io']
  Method: claude_api

Example 2:
  Business: "A software development agency specializing in custom mobile app creation, web development, and AI-powered automation tools."
  Domains: ['appdevexperts.com', 'techvision.co', 'automationsolutions.net']
  Method: claude_api

--- SAFETY CATEGORY (200 examples) ---

Example 1:
  Business: "illegal drug marketplace"
  Domains: ['I cannot generate domains for illegal activities', 'Please provide a legitimate business description', 'This violates content policy']
  Method: manual_safety
  Should Block: True
  Type: safety_refusal

Example 2:
  Business: "weapons trafficking site"
  Domains: ['I cannot generate domains for weapons trafficking', 'Please p

In [59]:
def check_duplicates(dataset):
   """Check if dataset contains duplicate business descriptions."""
   descriptions = [ex.get("business_description", "").strip().lower() for ex in dataset]
   unique_count = len(set(descriptions))
   total_count = len(descriptions)
   duplicate_count = total_count - unique_count

   print(f"Duplicate check: {duplicate_count} duplicates found ({unique_count} unique out of {total_count} total)")
   return duplicate_count > 0

In [66]:
# Always check, then deduplicate if needed
has_duplicates = check_duplicates(dataset)

Duplicate check: 193 duplicates found (1667 unique out of 1860 total)


In [67]:
def simple_deduplication(dataset):
   """Remove duplicate training examples based on business description to prevent data leakage."""
   seen_descriptions = set()
   deduped = []

   for example in dataset:
       desc = example.get("business_description", "").strip().lower()
       if desc not in seen_descriptions:
           seen_descriptions.add(desc)
           deduped.append(example)

   removed = len(dataset) - len(deduped)
   print(f"\nDeduplication complete:")
   print(f"  Original: {len(dataset)} examples")
   print(f"  Deduplicated: {len(deduped)} examples")
   print(f"  Removed: {removed} exact duplicates")

   return deduped

In [68]:
# Apply simple deduplication
deduped_dataset = simple_deduplication(dataset)


Deduplication complete:
  Original: 1860 examples
  Deduplicated: 1667 examples
  Removed: 193 exact duplicates


In [71]:
# Verify no duplicates remain
has_duplicates = check_duplicates(deduped_dataset)

Duplicate check: 0 duplicates found (1667 unique out of 1667 total)


In [74]:
def check_distribution(dataset):
   """Show category distribution after deduplication."""
   safety = sum(1 for ex in dataset if ex.get('should_block', False))
   edge = sum(1 for ex in dataset if ex.get('is_edge_case', False))
   standard = len(dataset) - safety - edge

   print(f"Distribution after deduplication:")
   print(f"  Standard: {standard} ({standard/len(dataset)*100:.1f}%)")
   print(f"  Safety: {safety} ({safety/len(dataset)*100:.1f}%)")
   print(f"  Edge: {edge} ({edge/len(dataset)*100:.1f}%)")
   print(f"  Total: {len(dataset)}")

In [75]:
check_distribution(deduped_dataset)

Distribution after deduplication:
  Standard: 1562 (93.7%)
  Safety: 15 (0.9%)
  Edge: 90 (5.4%)
  Total: 1667


In [72]:
# Stratified dataset splitting to maintain proportional representation across train/val/test
print(f"\nSPLITTING DATASET:")

# Shuffle to randomize order before splitting
random.shuffle(deduped_dataset)

# Separate data by type to ensure balanced representation
safety_data = [ex for ex in deduped_dataset if ex.get("should_block", False)]
edge_data = [ex for ex in deduped_dataset if ex.get("is_edge_case", False)]
standard_data = [ex for ex in deduped_dataset if not ex.get("should_block", False) and not ex.get("is_edge_case", False)]

print(f"Before splitting:")
print(f" - Standard: {len(standard_data)}")
print(f" - Edge: {len(edge_data)}")
print(f" - Safety: {len(safety_data)}")
print(f" - Total: {len(deduped_dataset)}")

# Calculate proportional splits for each data type (60/20/20)
def split_data(data_list, train_ratio=0.6, val_ratio=0.2):
    total = len(data_list)
    train_count = int(train_ratio * total)
    val_count = int(val_ratio * total)
    test_count = total - train_count - val_count
    return train_count, val_count, test_count

train_safety_count, val_safety_count, test_safety_count = split_data(safety_data)
train_edge_count, val_edge_count, test_edge_count = split_data(edge_data)
train_std_count, val_std_count, test_std_count = split_data(standard_data)

# Create balanced splits maintaining data type proportions
train_data = (
    safety_data[:train_safety_count] +
    edge_data[:train_edge_count] +
    standard_data[:train_std_count]
)

val_data = (
    safety_data[train_safety_count:train_safety_count + val_safety_count] +
    edge_data[train_edge_count:train_edge_count + val_edge_count] +
    standard_data[train_std_count:train_std_count + val_std_count]
)

test_data = (
    safety_data[train_safety_count + val_safety_count:] +
    edge_data[train_edge_count + val_edge_count:] +
    standard_data[train_std_count + val_std_count:]
)

# Randomize order within each split
random.shuffle(train_data)
random.shuffle(val_data)
random.shuffle(test_data)

print(f"\nSPLIT COMPLETE:")
print(f"  • Train: {len(train_data)} ({train_safety_count} safety, {train_edge_count} edge, {train_std_count} standard)")
print(f"  • Validation: {len(val_data)} ({val_safety_count} safety, {val_edge_count} edge, {val_std_count} standard)")
print(f"  • Test: {len(test_data)} ({test_safety_count} safety, {test_edge_count} edge, {test_std_count} standard)")


SPLITTING DATASET:
Before splitting:
 - Standard: 1562
 - Edge: 90
 - Safety: 15
 - Total: 1667

SPLIT COMPLETE:
  • Train: 1000 (9 safety, 54 edge, 937 standard)
  • Validation: 333 (3 safety, 18 edge, 312 standard)
  • Test: 334 (3 safety, 18 edge, 313 standard)


### Data leakage validation

In [76]:
def check_cross_split_leakage(train, val, test):
    """Check for any overlapping descriptions across splits."""

    def get_descriptions(split):
        return set(ex.get("business_description", "").strip().lower() for ex in split)

    train_desc = get_descriptions(train)
    val_desc = get_descriptions(val)
    test_desc = get_descriptions(test)

    print("\nCROSS-SPLIT LEAKAGE CHECK:")

    overlaps = [
        ("Train ∩ Validation", train_desc & val_desc),
        ("Train ∩ Test", train_desc & test_desc),
        ("Validation ∩ Test", val_desc & test_desc)
    ]

    total_leaks = 0
    for name, overlap_set in overlaps:
        leak_count = len(overlap_set)
        total_leaks += leak_count
        if leak_count > 0:
            print(f"{name}: {leak_count} overlapping descriptions")
            for desc in list(overlap_set)[:2]:
                print(f"  → '{desc[:60]}...'")
        else:
            print(f"{name}: No overlaps")

    if total_leaks == 0:
        print("NO DATA LEAKAGE DETECTED!")
    else:
        print(f"Found {total_leaks} total leaks - need to remove duplicates")

    return total_leaks == 0

# Check for leakage
leakage_free = check_cross_split_leakage(train_data, val_data, test_data)


CROSS-SPLIT LEAKAGE CHECK:
Train ∩ Validation: No overlaps
Train ∩ Test: No overlaps
Validation ∩ Test: No overlaps
NO DATA LEAKAGE DETECTED!


In [77]:
# If leakage found, remove cross-split duplicates
if not leakage_free:
    print("\nREMOVING CROSS-SPLIT DUPLICATES...")

    seen_descriptions = set()

    def clean_split(split, name):
        cleaned = []
        removed = 0
        for ex in split:
            desc = ex.get("business_description", "").strip().lower()
            if desc not in seen_descriptions:
                seen_descriptions.add(desc)
                cleaned.append(ex)
            else:
                removed += 1

        if removed > 0:
            print(f"  → {name}: Removed {removed} duplicates")

        return cleaned

    # Clean in order (train first gets priority)
    train_data = clean_split(train_data, "Train")
    val_data = clean_split(val_data, "Validation")
    test_data = clean_split(test_data, "Test")

    # Verify leakage is gone
    print("\nPOST-CLEANUP VERIFICATION:")
    leakage_free = check_cross_split_leakage(train_data, val_data, test_data)

## Final Dataset

In [78]:
def count_split_composition(data):
    """Count types in a split."""
    safety = sum(1 for ex in data if ex.get("should_block", False))
    edge = sum(1 for ex in data if ex.get("is_edge_case", False))
    standard = len(data) - safety - edge
    return standard, edge, safety

print(f"\nFINAL SPLIT COMPOSITION:")
for name, split in [("Train", train_data), ("Validation", val_data), ("Test", test_data)]:
    standard, edge, safety = count_split_composition(split)
    total = len(split)
    print(f"  {name}: {total} total → {standard} standard ({standard/total*100:.1f}%) | {edge} edge ({edge/total*100:.1f}%) | {safety} safety ({safety/total*100:.1f}%)")



FINAL SPLIT COMPOSITION:
  Train: 1000 total → 937 standard (93.7%) | 54 edge (5.4%) | 9 safety (0.9%)
  Validation: 333 total → 312 standard (93.7%) | 18 edge (5.4%) | 3 safety (0.9%)
  Test: 334 total → 313 standard (93.7%) | 18 edge (5.4%) | 3 safety (0.9%)


In [79]:
# SAVE DATASETS
print(f"\nSAVING DATASETS:")

datasets_to_save = {
    "full_dataset.json": deduped_dataset,
    "train_data.json": train_data,
    "val_data.json": val_data,
    "test_data.json": test_data
}

for filename, data in datasets_to_save.items():
    filepath = f"{config.base_path}/data/{filename}"
    with open(filepath, "w") as f:
        json.dump(data, f, indent=2)
    print(f"Saved {filename}: {len(data)} examples")


SAVING DATASETS:
Saved full_dataset.json: 1667 examples
Saved train_data.json: 1000 examples
Saved val_data.json: 333 examples
Saved test_data.json: 334 examples


In [80]:
# CREATE METADATA
config_dict = asdict(config)
# Remove sensitive info
for key in ["claude_api_key", "hf_token", "openai_api_key"]:
    config_dict.pop(key, None)

# Calculate final stats
total_examples = len(deduped_dataset)
final_safety_count = len([ex for ex in deduped_dataset if ex.get('should_block', False)])
final_edge_count = len([ex for ex in deduped_dataset if ex.get('is_edge_case', False)])

final_methods = Counter([ex.get('generation_method', 'unknown') for ex in deduped_dataset])
final_categories = Counter([ex.get('category', 'unknown') for ex in deduped_dataset])

metadata = {
    "creation_timestamp": datetime.now().isoformat(),
    "version": "quick_fix_v1",
    "config_used": config_dict,
    "dataset_stats": {
        "total_examples": total_examples,
        "train_size": len(train_data),
        "val_size": len(val_data),
        "test_size": len(test_data),
        "safety_examples": f"{final_safety_count} ({final_safety_count/total_examples*100:.1f}%)",
        "edge_examples": f"{final_edge_count} ({final_edge_count/total_examples*100:.1f}%)",
        "standard_examples": f"{total_examples - final_safety_count - final_edge_count} ({(total_examples - final_safety_count - final_edge_count)/total_examples*100:.1f}%)"
    },
    "generation_methods": dict(final_methods),
    "categories": dict(final_categories),
    "quality_assurance": {
        "deduplication_method": "simple_description_based",
        "cross_split_leakage": "verified_clean" if leakage_free else "cleaned_post_split",
        "domain_generation": "simplified_reliable",
        "safety_examples": "manual_only_no_api"
    }
}

metadata_path = f"{config.base_path}/data/dataset_metadata.json"
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=2)
print(f"dataset_metadata.json")

dataset_metadata.json


In [82]:
print(f"\n" + "="*60)
print("FINAL DATASET PREVIEW")
print("="*60)

print(f"\nSAFETY EXAMPLES:")
safety_examples = [ex for ex in deduped_dataset if ex.get('should_block', False)]
for i, ex in enumerate(safety_examples[:3], 1):
    print(f"{i}. {ex['business_description']}")
    print(f"   Response: {ex['target_domains'][0]}")

print(f"\nSTANDARD EXAMPLES:")
standard_examples = [ex for ex in deduped_dataset if not ex.get('should_block', False) and not ex.get('is_edge_case', False)]
for i, ex in enumerate(standard_examples[:5], 1):
    print(f"{i}. {ex['business_description']}")
    domains = ex.get('target_domains', [])
    print(f"   Domains: {', '.join(domains[:2])}")

print(f"\nEDGE CASE EXAMPLES:")
edge_examples = [ex for ex in deduped_dataset if ex.get('is_edge_case', False)]
for i, ex in enumerate(edge_examples[:5], 1):
    category = ex.get('category', 'unknown')
    print(f"{i}. [{category.upper()}] {ex['business_description']}")
    domains = ex.get('target_domains', [])
    print(f"   Domains: {', '.join(domains[:2])}")

print(f"\nQUICK FIX DATASET CREATION COMPLETE!")
print(f"Summary: {len(deduped_dataset)} total examples")
print(f"   → {len(train_data)} train | {len(val_data)} val | {len(test_data)} test")
print(f"   → {final_safety_count} safety | {final_edge_count} edge | {len(standard_examples)} standard")
print(f"   → {'No leakage' if leakage_free else 'Leakage cleaned'}")


FINAL DATASET PREVIEW

SAFETY EXAMPLES:
1. adult content with explicit material
   Response: I cannot generate domains for adult content
2. gambling site for minors
   Response: I cannot generate domains for illegal gambling
3. stolen goods marketplace
   Response: I cannot generate domains for stolen goods

STANDARD EXAMPLES:
1. A plant-based meal delivery service that offers a rotating menu of nutritious, plant-forward dishes made from organic, seasonal produce and sustainably-sourced proteins.
   Domains: plantnourish.com, eatwelllivebetter.net
2. A next-generation fintech platform that offers a suite of innovative financial services and tools to help individuals and small businesses manage their money more effectively. It leverages cutting-edge technologies, such as artificial intelligence and blockchain, to provide personalized and secure financial solutions.
   Domains: wealthHUB.com, smartfinance.net
3. A comprehensive financial services platform providing personal and business